# 프롬프트를 위한 가이드라인
이 레슨에서는 대규모 언어 모델에 효과적인 프롬프트를 작성하기 위해 두 가지 프롬프트 원칙과 관련 전략을 연습합니다.

## 설정
#### API 키와 관련 Python 라이브러리를 로드합니다.

이 과정에서는 OpenAI API 키를 로드하는 몇 가지 코드를 제공했습니다.

In [2]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key  = os.getenv('OPENAI_API_KEY')

#### helper function
이 과정에서는 OpenAI의 `gpt-3.5-turbo` 모델과 [채팅 완료 엔드포인트](https://platform.openai.com/docs/guides/chat)를 사용할 것입니다. 

이 helper function을 사용하면 프롬프트를 더 쉽게 사용하고 생성된 출력을 확인할 수 있습니다:

In [3]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

## 프롬프트 원칙
- 원칙 1: 명확하고 구체적인 지침을 작성하세요**.
- 원칙 2: 모델에게 '생각할 시간'을 주세요**.

### 전술

#### 전술 1: 구분 기호를 사용하여 입력의 뚜렷한 부분을 명확하게 표시하세요.
- 구분 기호는 다음과 같이 사용할 수 있습니다: ```, """, < >, `<태그> </태그>`, `:` 등입니다.

In [5]:
text = f"""
You should express what you want a model to do by \ 
providing instructions that are as clear and \ 
specific as you can possibly make them. \ 
This will guide the model towards the desired output, \ 
and reduce the chances of receiving irrelevant \ 
or incorrect responses. Don't confuse writing a \ 
clear prompt with writing a short prompt. \ 
In many cases, longer prompts provide more clarity \ 
and context for the model, which can lead to \ 
more detailed and relevant outputs.
"""
prompt = f"""
Summarize the text delimited by triple backticks \ 
into a single sentence.
```{text}```
"""
response = get_completion(prompt)
print(response)

Clear and specific instructions should be provided to guide a model towards the desired output, and longer prompts can provide more clarity and context for the model, leading to more detailed and relevant outputs.


In [6]:
text = f"""
You should express what you want a model to do by \ 
providing instructions that are as clear and \ 
specific as you can possibly make them. \ 
This will guide the model towards the desired output, \ 
and reduce the chances of receiving irrelevant \ 
or incorrect responses. Don't confuse writing a \ 
clear prompt with writing a short prompt. \ 
In many cases, longer prompts provide more clarity \ 
and context for the model, which can lead to \ 
more detailed and relevant outputs.
"""
prompt = f"""
Summarize the text delimited by triple backticks \ 
into a single sentence. Response in Korean.
```{text}```
"""
response = get_completion(prompt)
print(response)

모델이 원하는 출력물을 만들기 위해서는 명확하고 구체적인 지시사항을 제공하여 모델을 이끌어야 하며, 이는 불필요하거나 잘못된 응답을 받는 가능성을 줄일 수 있으며, 명확한 프롬프트와 짧은 프롬프트를 혼동하지 말아야 하며, 많은 경우 더 긴 프롬프트가 모델에게 더 많은 명확성과 문맥을 제공하여 더 자세하고 관련성 높은 출력물을 만들어 낼 수 있다.


#### 전술 2: 구조화된 출력 요청하기
- JSON, HTML

In [7]:
prompt = f"""
Generate a list of three made-up book titles along \ 
with their authors and genres.
Provide them in JSON format with the following keys: 
book_id, title, author, genre.
"""
response = get_completion(prompt)
print(response)

[
  {
    "book_id": 1,
    "title": "The Lost City of Zorath",
    "author": "Aria Blackwood",
    "genre": "Fantasy"
  },
  {
    "book_id": 2,
    "title": "The Last Survivors",
    "author": "Ethan Stone",
    "genre": "Science Fiction"
  },
  {
    "book_id": 3,
    "title": "The Secret of the Haunted Mansion",
    "author": "Lila Rose",
    "genre": "Mystery"
  }
]


In [8]:
prompt = f"""
Generate a list of three made-up book titles along \ 
with their authors and genres. Answer in Korean.
Provide them in JSON format with the following keys: 
book_id, title, author, genre.
"""
response = get_completion(prompt)
print(response)

{
    "book1": {
        "title": "불멸의 꽃",
        "author": "김지영",
        "genre": "판타지"
    },
    "book2": {
        "title": "그림자의 노래",
        "author": "이승희",
        "genre": "로맨스"
    },
    "book3": {
        "title": "바다의 눈물",
        "author": "박지현",
        "genre": "모험"
    }
}


#### 전술 3: 모델에게 조건이 충족되었는지 확인하도록 요청하기

In [9]:
text_1 = f"""
Making a cup of tea is easy! First, you need to get some \ 
water boiling. While that's happening, \ 
grab a cup and put a tea bag in it. Once the water is \ 
hot enough, just pour it over the tea bag. \ 
Let it sit for a bit so the tea can steep. After a \ 
few minutes, take out the tea bag. If you \ 
like, you can add some sugar or milk to taste. \ 
And that's it! You've got yourself a delicious \ 
cup of tea to enjoy.
"""
prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, \ 
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \ 
then simply write \"No steps provided.\"

\"\"\"{text_1}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Text 1:")
print(response)

Completion for Text 1:
Step 1 - Get some water boiling.
Step 2 - Grab a cup and put a tea bag in it.
Step 3 - Once the water is hot enough, pour it over the tea bag.
Step 4 - Let it sit for a bit so the tea can steep.
Step 5 - After a few minutes, take out the tea bag.
Step 6 - Add some sugar or milk to taste.
Step 7 - Enjoy your delicious cup of tea!




In [14]:
text_1 = f"""
Making a cup of tea is easy! First, you need to get some \ 
water boiling. While that's happening, \ 
grab a cup and put a tea bag in it. Once the water is \ 
hot enough, just pour it over the tea bag. \ 
Let it sit for a bit so the tea can steep. After a \ 
few minutes, take out the tea bag. If you \ 
like, you can add some sugar or milk to taste. \ 
And that's it! You've got yourself a delicious \ 
cup of tea to enjoy.
"""
prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, \ 
re-write those instructions in the following format using Korean:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \ 
then simply write \"제공된 단계 없음.\"

\"\"\"{text_1}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Text 1:")
print(response)

Completion for Text 1:
단계 1 - 물을 끓이세요.
단계 2 - 컵을 가져와서 차백을 넣으세요.
단계 3 - 물이 충분히 끓으면 차백 위에 붓으세요.
단계 4 - 차가 우려질 때까지 잠시 기다리세요.
단계 5 - 차백을 꺼내세요.
단계 6 - 원하신다면 설탕이나 우유를 추가하세요.
그리고 이제 맛있는 차 한잔이 완성되었습니다!


In [12]:
text_2 = f"""
The sun is shining brightly today, and the birds are \
singing. It's a beautiful day to go for a \ 
walk in the park. The flowers are blooming, and the \ 
trees are swaying gently in the breeze. People \ 
are out and about, enjoying the lovely weather. \ 
Some are having picnics, while others are playing \ 
games or simply relaxing on the grass. It's a \ 
perfect day to spend time outdoors and appreciate the \ 
beauty of nature.
"""
prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, \ 
re-write those instructions in the following format using Korean:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \ 
then simply write \"제공된 단계 없음.\"

\"\"\"{text_2}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Text 2:")
print(response)

Completion for Text 2:
제공된 단계 없음.


#### 전술 4: "Few-shot" 프롬프트

In [16]:
prompt = f"""
Your task is to answer in a consistent style. 

<child>: Teach me about patience.

<grandparent>: The river that carves the deepest \ 
valley flows from a modest spring; the \ 
grandest symphony originates from a single note; \ 
the most intricate tapestry begins with a solitary thread.

<child>: Teach me about resilience.
"""
response = get_completion(prompt)
print(response)

<grandparent>: Resilience is like a tree that bends with the wind but never breaks. It is the ability to bounce back from adversity and keep moving forward, even when the road ahead is tough. Just like a tree that grows stronger with each storm it weathers, resilience is a quality that can be developed and strengthened over time.


In [15]:
prompt = f"""
Your task is to answer in a consistent style. Using Korean.

<child>: Teach me about patience.

<grandparent>: 가장 깊은 \ 
계곡은 겸손한 샘에서 흐르고, 가장 \ 
가장 웅장한 교향곡은 하나의 음에서 시작됩니다. \
가장 복잡한 태피스트리는 고독한 실에서 시작됩니다.

<child>: Teach me about resilience.
"""
response = get_completion(prompt)
print(response)

<grandparent>: 태양이 떠오르는 동안 꽃은 굳건하게 자신을 지켜내며, 비가 내리는 동안 나무는 강한 뿌리로 인해 굳건하게 서있습니다. 인생에서도 어려움이 찾아와도 우리는 굳건하게 버티며 살아갈 수 있습니다.


### 원칙 2: 모델에게 "생각할" 시간을 주세요. 

#### 전략 1: 작업을 완료하는 데 필요한 단계를 지정하세요.

In [17]:
text = f"""
In a charming village, siblings Jack and Jill set out on \ 
a quest to fetch water from a hilltop \ 
well. As they climbed, singing joyfully, misfortune \ 
struck—Jack tripped on a stone and tumbled \ 
down the hill, with Jill following suit. \ 
Though slightly battered, the pair returned home to \ 
comforting embraces. Despite the mishap, \ 
their adventurous spirits remained undimmed, and they \ 
continued exploring with delight.
"""
# example 1
prompt_1 = f"""
Perform the following actions: 
1 - Summarize the following text delimited by triple \
backticks with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the following \
keys: french_summary, num_names.

Separate your answers with line breaks.

Text:
```{text}```
"""
response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:
Two siblings, Jack and Jill, go on a quest to fetch water from a hilltop well, but misfortune strikes as they both fall down the hill, yet they return home slightly battered but with their adventurous spirits undimmed.

Deux frères et sœurs, Jack et Jill, partent en quête d'eau d'un puits au sommet d'une colline, mais ils tombent tous les deux et retournent chez eux légèrement meurtris mais avec leur esprit d'aventure intact. 
Noms: Jack, Jill.

{
"french_summary": "Deux frères et sœurs, Jack et Jill, partent en quête d'eau d'un puits au sommet d'une colline, mais ils tombent tous les deux et retournent chez eux légèrement meurtris mais avec leur esprit d'aventure intact.",
"num_names": 2
}


In [18]:
text = f"""
In a charming village, siblings Jack and Jill set out on \ 
a quest to fetch water from a hilltop \ 
well. As they climbed, singing joyfully, misfortune \ 
struck—Jack tripped on a stone and tumbled \ 
down the hill, with Jill following suit. \ 
Though slightly battered, the pair returned home to \ 
comforting embraces. Despite the mishap, \ 
their adventurous spirits remained undimmed, and they \ 
continued exploring with delight.
"""
# example 1
prompt_1 = f"""
Perform the following actions: 
1 - Summarize the following text delimited by triple \
backticks with 1 sentence.
2 - Translate the summary into Korean.
3 - List each name in the Korean summary.
4 - Output a json object that contains the following \
keys: korean_summary, num_names.

Separate your answers with line breaks.

Text:
```{text}```
"""
response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:
1 - Siblings Jack and Jill go on a quest to fetch water from a hilltop well, but misfortune strikes as Jack trips and tumbles down the hill, with Jill following suit, yet they return home slightly battered but with their adventurous spirits undimmed. 

2 - 자매 형제인 Jack과 Jill은 언덕 꼭대기에 있는 우물에서 물을 길어오기 위해 여행을 떠난다. 그러나 Jack이 돌에 걸려 언덕을 굴러내려 Jill도 따라가며 불행이 닥치지만, 그들은 약간 상처를 입고 집으로 돌아가지만 그들의 모험적인 정신은 여전히 유지되며 기쁨으로 탐험을 계속한다.

3 - Jack, Jill

4 - {
      "korean_summary": "자매 형제인 Jack과 Jill은 언덕 꼭대기에 있는 우물에서 물을 길어오기 위해 여행을 떠난다. 그러나 Jack이 돌에 걸려 언덕을 굴러내려 Jill도 따라가며 불행이 닥치지만, 그들은 약간 상처를 입고 집으로 돌아가지만 그들의 모험적인 정신은 여전히 유지되며 기쁨으로 탐험을 계속한다.",
      "num_names": 2
   }


#### 지정된 형식으로 출력 요청하기

In [19]:
prompt_2 = f"""
Your task is to perform the following actions: 
1 - Summarize the following text delimited by 
  <> with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the 
  following keys: french_summary, num_names.

Use the following format:
Text: <text to summarize>
Summary: <summary>
Translation: <summary translation>
Names: <list of names in Italian summary>
Output JSON: <json with summary and num_names>

Text: <{text}>
"""
response = get_completion(prompt_2)
print("\nCompletion for prompt 2:")
print(response)


Completion for prompt 2:
Summary: Jack and Jill go on a quest to fetch water, but misfortune strikes and they tumble down the hill, returning home slightly battered but with their adventurous spirits undimmed. 
Translation: Jack et Jill partent en quête d'eau, mais la malchance frappe et ils dégringolent la colline, rentrant chez eux légèrement meurtris mais avec leurs esprits aventureux intacts.
Names: Jack, Jill
Output JSON: {"french_summary": "Jack et Jill partent en quête d'eau, mais la malchance frappe et ils dégringolent la colline, rentrant chez eux légèrement meurtris mais avec leurs esprits aventureux intacts.", "num_names": 2}


In [20]:
prompt_2 = f"""
Your task is to perform the following actions: 
1 - Summarize the following text delimited by 
  <> with 1 sentence.
2 - Translate the summary into Korean.
3 - List each name in the Korean summary.
4 - Output a json object that contains the 
  following keys: korean_summary, num_names.

Use the following format:
Text: <text to summarize>
Summary: <summary>
Translation: <summary translation>
Names: <list of names in Korean summary>
Output JSON: <json with summary and num_names>

Text: <{text}>
"""
response = get_completion(prompt_2)
print("\nCompletion for prompt 2:")
print(response)


Completion for prompt 2:
Summary: Jack and Jill go on a quest to fetch water, but misfortune strikes and they tumble down a hill, returning home slightly battered but with undimmed adventurous spirits. 
Translation: 잭과 질은 물을 가져 오기 위해 여행을 떠납니다. 그러나 불행이 닥쳐서 그들은 언덕을 굴러 내려가고, 약간 상처를 입고 집으로 돌아갑니다. 그러나 그들의 모험적인 정신은 여전히 활발하며 기쁨으로 탐험을 계속합니다.
Names: 잭, 질 (Jack, Jill)
Output JSON: {"korean_summary": "잭과 질은 물을 가져 오기 위해 여행을 떠납니다. 그러나 불행이 닥쳐서 그들은 언덕을 굴러 내려가고, 약간 상처를 입고 집으로 돌아갑니다. 그러나 그들의 모험적인 정신은 여전히 활발하며 기쁨으로 탐험을 계속합니다.", "num_names": 2}


#### 전술 2: 결론을 내리기 전에 모델 스스로 해결책을 찾도록 지시하세요.

In [21]:
prompt = f"""
Determine if the student's solution is correct or not.

Question:
I'm building a solar power installation and I need \
 help working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \ 
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations 
as a function of the number of square feet.

Student's Solution:
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
"""
response = get_completion(prompt)
print(response)

The student's solution is correct.


#### 학생의 솔루션이 실제로는 올바르지 않습니다.
#### 모델에 먼저 자체적으로 해결책을 찾도록 지시하여 이 문제를 해결할 수 있습니다.

In [22]:
prompt = f"""
Your task is to determine if the student's solution \
is correct or not.
To solve the problem do the following:
- First, work out your own solution to the problem. 
- Then compare your solution to the student's solution \ 
and evaluate if the student's solution is correct or not. 
Don't decide if the student's solution is correct until 
you have done the problem yourself.

Use the following format:
Question:
```
question here
```
Student's solution:
```
student's solution here
```
Actual solution:
```
steps to work out the solution and your solution here
```
Is the student's solution the same as actual solution \
just calculated:
```
yes or no
```
Student grade:
```
correct or incorrect
```

Question:
```
I'm building a solar power installation and I need help \
working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations \
as a function of the number of square feet.
``` 
Student's solution:
```
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
```
Actual solution:
"""
response = get_completion(prompt)
print(response)

Let x be the size of the installation in square feet.

Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 10x

Total cost: 100x + 250x + 100,000 + 10x = 360x + 100,000

Is the student's solution the same as actual solution just calculated:
No

Student grade:
Incorrect


## 모델 제한 사항: 환각
- Boie는 실제 회사이며 제품 이름은 실제가 아닙니다.

In [23]:
prompt = f"""
Tell me about AeroGlide UltraSlim Smart Toothbrush by Boie
"""
response = get_completion(prompt)
print(response)

The AeroGlide UltraSlim Smart Toothbrush by Boie is a high-tech toothbrush that uses advanced sonic technology to provide a deep and thorough clean. It features a slim and sleek design that makes it easy to hold and maneuver, and it comes with a range of smart features that help you optimize your brushing routine.

One of the key features of the AeroGlide UltraSlim Smart Toothbrush is its advanced sonic technology, which uses high-frequency vibrations to break up plaque and bacteria on your teeth and gums. This technology is highly effective at removing even the toughest stains and buildup, leaving your teeth feeling clean and refreshed.

In addition to its sonic technology, the AeroGlide UltraSlim Smart Toothbrush also comes with a range of smart features that help you optimize your brushing routine. These include a built-in timer that ensures you brush for the recommended two minutes, as well as a pressure sensor that alerts you if you're brushing too hard.

Overall, the AeroGlide Ul

In [24]:
prompt = f"""
Tell me about AeroGlide UltraSlim Smart Toothbrush by Boie in Korean
"""
response = get_completion(prompt)
print(response)

AeroGlide UltraSlim Smart Toothbrush by Boie는 미국의 칫솔 브랜드인 Boie에서 출시한 스마트 칫솔입니다. 이 제품은 초슬림한 디자인으로 구성되어 있어 어디서든 쉽게 사용할 수 있습니다. 또한, 이 제품은 블루투스 기능을 탑재하여 스마트폰과 연동하여 사용할 수 있습니다. 이를 통해 사용자는 칫솔 사용 시간, 세척 횟수 등을 모니터링할 수 있습니다. 또한, 이 제품은 부드러운 실리콘 소재로 만들어져 치아와 잇몸을 부드럽게 마사지하며 치아의 표면을 깨끗하게 닦아줍니다. 이 제품은 USB 충전이 가능하며, 한 번 충전으로 최대 30일간 사용할 수 있습니다.


## 직접 실험해 보세요!

#### 이 강의실 외부에서 OpenAI API를 사용할 때 참고 사항

OpenAI 파이썬 라이브러리를 설치하려면
```
!pip install openai
```

라이브러리는 [웹사이트](https://platform.openai.com/account/api-keys)에서 확인할 수 있는 계정의 비밀 키로 구성해야 합니다. 

라이브러리를 사용하기 전에 `OPENAI_API_KEY` 환경 변수로 설정할 수 있습니다:
 ```
 !export OPENAI_API_KEY='sk-...'
 ```

또는 `openai.api_key`를 해당 값으로 설정합니다:

```
import openai
openai.api_key = "sk-..."
```

#### 백슬래시에 대한 참고 사항
- 이 강좌에서는 백슬래시 `\`를 사용하여 개행 '\n' 문자를 삽입하지 않고도 텍스트가 화면에 맞도록 하고 있습니다.
- GPT-3는 개행 문자를 삽입하든 삽입하지 않든 별다른 영향을 받지 않습니다.  그러나 일반적으로 LLM으로 작업할 때는 프롬프트의 개행 문자가 모델의 성능에 영향을 미칠 수 있는지 고려할 수 있습니다.